In [1]:
from occupancy.pipelines.autoencoderkl_3d import AutoEncoderKL3d, AutoEncoderKL3dConfig
import torch
import math
import logging
from torch import nn, Tensor, onnx
vae3d = AutoEncoderKL3d(1, 1, 16, exportable=True)
vae3d.load_state_dict(torch.load('/home/zc2309/workspace/occupancy/models/autoencoderkl-cls1.pt'))

<All keys matched successfully>

In [1]:
256 * 256 * 32 / 1000000

2.097152

In [ ]:
def occ_shuffle(occ: t)

In [3]:
from typing import Optional


tensor_x = torch.rand(1, 1, 256, 256, 32).float()
tensor_y = torch.rand(1, 16, 32, 32, 4).float()
vae3d = vae3d.float()
encoder_onnx = onnx.dynamo_export(vae3d.encoder, tensor_x)
decoder_onnx = onnx.dynamo_export(vae3d.decoder, tensor_y)

In [ ]:
encoder_onnx.save('autoencoderkl-cls1-encoder.onnx')
decoder_onnx.save('autoencoderkl-cls1-decoder.onnx')

In [ ]:
32 * 32 * 32

In [4]:
import torch
import sys
import os
sys.path.append(os.path.dirname(__file__))
torch.hub.load('serizba/salad', 'dinov2_salad')

NameError: name '__file__' is not defined